# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122031e+02     1.605891e+00
 * time: 0.09089398384094238
     1     1.094251e+01     8.371706e-01
 * time: 0.9229190349578857
     2    -1.215395e+01     1.021238e+00
 * time: 1.109787940979004
     3    -3.451538e+01     7.633220e-01
 * time: 1.2514488697052002
     4    -4.806141e+01     5.944475e-01
 * time: 1.4167988300323486
     5    -5.705667e+01     1.861962e-01
 * time: 1.569464921951294
     6    -5.985841e+01     9.755490e-02
 * time: 1.6795730590820312
     7    -6.090483e+01     5.510798e-02
 * time: 1.7990589141845703
     8    -6.129507e+01     6.613868e-02
 * time: 1.9097979068756104
     9    -6.160966e+01     3.337990e-02
 * time: 2.028977870941162
    10    -6.181147e+01     2.511398e-02
 * time: 2.140645980834961
    11    -6.197857e+01     1.859732e-02
 * time: 2.2588050365448
    12    -6.204903e+01     1.519457e-02
 * time: 2.3675689697265625
    13    -6.211840e+01     1.589424e-02
 * time: 2.4830868244171143

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557682
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336821

    total               -62.261666460374
